# Tutorial 02

Agenda:
- Filtros Espaciales
- Edge Detection

# Setup

Este tutorial se puede ejecutar local en Jupyter lab o utilizar Google Colab.

## En Google Colab 
Este tutorial se provee junto con archivos de recursos dentro de un archivo ".zip".
En caso de ejecutar en Google Colab hay que:

1. Descomprimir el zip en algún lado
2. Subir el contenido del zip a Google Drive en alguna carpeta (por ejemplo `udesa/I308/tutoriales/tutorial_X`)
3. Abrir este notebook .ipynb 

In [ ]:
import os
import sys

# TODO: establecer el path en caso de trabajar con Colab
DRIVE_DIR = "udesa/I308/tutoriales/tutorial_02"

# detecta si estamos corriendo en Google Colab
try:
  from google.colab import drive
  COLAB = True
except:
  COLAB = False

if COLAB:
    # monta Google Drive
    drive.mount('/content/drive')

    base_path = "/content/drive/MyDrive/"
    path = os.path.join(base_path, DRIVE_DIR)
    
    %cd {path}
    sys.path.append(path)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# instalamos el paquete de utilidades
!pip install -qq git+https://github.com/udesa-vision/i308-utils.git

from i308_utils import imshow, show_images

# Filtros Espaciales

## Padding de imagenes

La función de OpenCV [`cv2.copyMakeBorder`](https://docs.opencv.org/4.x/d2/de8/group__core__array.html#ga2ac1049c2c3dd25c2b41bffe17658a36) nos permite hacer
padding de imagenes.

Dependiendo del flag que se utiliza es el método que aplicará la función para realizar el padding (BORDER_CONSTANT, BORDER_REPLICATE, BORDER_REFLECT, etc).


In [ ]:
import cv2
img = cv2.imread("res/lenna.png", cv2.IMREAD_GRAYSCALE)
imshow(img)

In [ ]:
# ¿Cómo podemos agregar padding?
import numpy as np

def pad_image(img, border, color=0):
    
    h, w = img.shape
    shape = np.array((h, w))
    ret = color * np.ones(shape + 2 * border)
    ret[border:h+border, border:w+border] = img
    return ret

In [ ]:
imshow(pad_image(img, 128))

In [ ]:
# TODO:
# - usando la funcion de OpenCV aplicar padding a la imagen
# - variar los distintos tipos de bordes soportados
# - graficar los resultados
# - explicar brevemente la diferencia entre los distintos métodos

In [ ]:
top, bottom, left, right = [128] * 4

show_images([
  #img,
  cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=0),
  cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_REPLICATE),
  cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_WRAP),
  cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_REFLECT),
  # cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_REFLECT_101),

], [
    #"original",
    "BORDER_CONSTANT",
    "BORDER_REPLICATE",
    "BORDER_WRAP",
    "BORDER_REFLECT",
    # "BORDER_REFLECT_101",
])

In [ ]:
# | 1 2 3 | 2 1

In [ ]:
import numpy as np

# Creemos dos imágenes w de (3x3) y f de (5x5)
w = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
], dtype='float')

f = np.zeros((5, 5), dtype='float')
# f[2, 2] = 1
f[2:, 2:] = 1


In [ ]:
w

In [ ]:
#w = cv2.flip(w, -1)
#w

In [ ]:
f

In [ ]:
imshow(f)

## Convolución

La convolución modela la interacción entre dos señales para generar una tercer señal resultante. Ver [ejemplo 1D](https://upload.wikimedia.org/wikipedia/commons/6/6a/Convolution_of_box_signal_with_itself2.gif). 

En el contexto de procesamiento digital de imágenes, nuestras señales son discretas y están en 2D. Ver [ejemplo](https://upload.wikimedia.org/wikipedia/commons/1/19/2D_Convolution_Animation.gif) [.](https://github.com/udesa-vision/i308-resources/blob/main/tutoriales/tutorial_02/conv2d.png?raw=true)  

Implementar en python la función una funcion que realiza convolución, dadas dos dos imágenes w y f en escala de grises:

$ (w ★ f)( x, y) =  \sum\limits_{s=-∞}^{∞} \sum\limits_{t=-∞}^{∞} w(s, t) . f(x - s, y - t ) $

Notas:
- El kernel $w$ está indexado en base a un punto central (anchor), por lo que tipicamente las dimensiones del kernel son impares.
- $w$ y $f$ se extienden de forma tal que valgan ceros por fuera de los límites de la imagen.

In [ ]:
# - Implementar en pyhton la función convolution que recibe un kernel w, y una imagen f
#   y devuelve la convolucion w * f.
#   para este ejercicio no importa la eficiencia, se pueden usar for loops.
#  **hint:** revisar la función de opencv `cv2.copyMakeBorder` que permite
#  hacer padding de imagenes.

import numpy as np
import cv2

def convolution(w, f):
    
    M, N = f.shape       # tamaño de la imagen
    m, n = w.shape       # tamaño del kernel
    a = m // 2           # offset vertical
    b = n // 2           # offset horizontal

    # aplica zero padding
    f_padded = cv2.copyMakeBorder(f, a, a, b, b, cv2.BORDER_CONSTANT)

    g = np.zeros_like(f, dtype=np.float64)
    
    for y in range(M):
        for x in range(N):
            v = 0
            for s in range(-a, a + 1):
                for t in range(-b, b + 1):
                    v += w[s + a, t + b] * f_padded[y - s + a, x - t + b]
            g[y, x] = v
    
    return g


In [ ]:
h_conv = convolution(w, f)

In [ ]:
h_conv

In [ ]:
imshow(h_conv)

In [ ]:
# podemos pensar w y f como sabanas infinitas de ceros con informacion localizada en una pequeña ventana.
w_pad = pad_image(w, 3)
f_pad = pad_image(f, 3)
show_images([
    w_pad,
    f_pad,
    convolution(w_pad, f_pad),
    # cv_convolution(w_pad, f_pad)
])

## Correlación

La función de correlación dadas dos dos imágenes w y f en escala de grises:

$ (w ☆ f)( x, y) =  \sum\limits_{s=-∞}^{∞} \sum\limits_{t=-∞}^{∞} w(s, t) . f(x + s, y + t ) $




In [ ]:
# Implementar en python una funcion correlation que recibe dos imágenes
#  en escala de grises w y f y devuelve la correlación.
# - para este ejercicio no importa la eficiencia, se pueden usar for loops.

def correlation(w, f, padding='zero'):
    M, N = f.shape
    m, n = w.shape
    a, b = m // 2, n // 2

    # aplicar padding simétrico
    f_padded = cv2.copyMakeBorder(f, a, a, b, b, cv2.BORDER_CONSTANT)

    g = np.zeros_like(f, dtype=np.float64)
    for y in range(M):
        for x in range(N):
            g[y, x] = (w * f_padded[y:y + m, x:x + n]).sum()

    return g


In [ ]:
h_corr = correlation(w, f)

In [ ]:
h_corr

In [ ]:
imshow(h_corr)

In [ ]:
# dan igual?
np.linalg.norm(h_corr - h_conv)

In [ ]:
# Propiedad: 
# convolution(w, f) == correlation(flip(w), f)

In [ ]:
w

In [ ]:
w_flip = cv2.flip(w, -1)
w_flip

In [ ]:
h_corr = correlation(w_flip, f)

In [ ]:
np.linalg.norm(h_corr - h_conv)

In [ ]:
# => si w es simétrico, correlation(w, f) == convolution(w, f)

## cv2.filter2D
la función de OpenCV [`cv2.filter2D`](https://docs.opencv.org/3.4/d4/dbd/tutorial_filter_2d.html) permite computar la correlación dado el kernel w, y la imagen f, de manera eficiente.


In [ ]:
# TODO:
# Usando OpenCV cv2.filter2D, computar la misma convolución que antes w * f.

def cv_convolution(w, f):
    
    return cv2.filter2D(
        f,
        ddepth=cv2.CV_64F,
        kernel=cv2.flip(w, -1),
        borderType=cv2.BORDER_CONSTANT
    )

cv_convolution(w, f)

In [ ]:
convolution(w, f)

In [ ]:
c1 = cv_convolution(w, f)
c2 = convolution(w, f)

In [ ]:
show_images([c1, c2])

In [ ]:
eq = np.linalg.norm(c1 - c2) < 0.001
assert eq, "el resultado de ambas funciones deben coincidir"

## Propiedades de la convolución

Verificar las siguientes propiedades de la convolución:

- **conmutativa**: (f ★ g) = (g ★ f)

- **asociativa**: (f ★ g) ★ h = f ★ (g ★ h)

- **distributiva**: f ★ (g + h) = (f ★ g) + (f ★ h)


In [ ]:
# Conmutativa
# w * f = f * w
# convolution(w, f) == convolution(f, w)

def pad(img, a, b):
  return cv2.copyMakeBorder(img, a, a, b, b, borderType=cv2.BORDER_CONSTANT, value=0)

w_pad = pad(w, 1, 1)

cv_convolution(w_pad, f)

In [ ]:
cv_convolution(f, w_pad)

In [ ]:
# Asociativa

h = np.array([
    [-1, 1, -1],
    [1, -8, 1],
    [-1, 1, -1],
], dtype='float')

# Propiedad:
# (w * f) * h = w * (f * h)
# convolution(convolution(w, f), h) == convolution(w, convolution(f, h))


In [ ]:
w_pad = pad_image(w, 2)
f_pad = pad_image(f, 2)
h_pad = pad_image(h, 2)


In [ ]:
convolution(convolution(w_pad, f_pad), h_pad)

In [ ]:
convolution(w_pad, convolution(f_pad, h_pad))

In [ ]:
# Distributiva

# Propiedad
# w * (f + h) = w * f + w * h
# convolution(w, (f + h)) == convolution(w, f) + convolution(w, h)

In [ ]:
h_pad = pad_image(h, 1)

In [ ]:
convolution(w, f + h_pad)

In [ ]:
convolution(w, f) + convolution(w, h_pad)

# Suavizado (Blur)

Smoothing o Blurring

A estos filtros se los denomina "pasa bajo" porque en la señal resultante persisten las bajas frecuencias.



### Filtro de Caja

El kernel caja (o box kernel) es un filtro lineal de dominio espacial en el que cada píxel de la imagen resultante tiene un valor igual al valor medio de sus píxeles vecinos en la imagen de entrada.

Por ejemplo, en el caso de 3x3 tenemos:

$
w = \frac{1}{9} .
  \begin{bmatrix}
    1 & 1 & 1 \\
    1 & 1 & 1 \\
    1 & 1 & 1
  \end{bmatrix}
$

In [ ]:
# (box blur)

# Escribir en python una función que dado el tamaño del kernel
# genere un kernel caja.
# - Luego aplicarlo como filtro sobre la imagen y verificar el resultado.
# - Que ocurre cuando el tamaño del kernel es más grande?

def box_kernel(size):
    
    k = 1 / (size * size) * np.ones((size, size), dtype='float')
    
    return k

labels = []
sizes = [3, 7, 11, 27]
results = [img]
labels = ["original"]

for size in sizes:

  k = box_kernel(size)

  filtered = cv2.filter2D(
      img, 
      kernel=k, 
      ddepth=cv2.CV_64F, 
      borderType=cv2.BORDER_CONSTANT
  )

  labels.append(f"box filter s:{size}")
  results.append(filtered)


show_images(results, labels)



### Filtro gaussiano


La [distribución de Gauss (o Normal) en dos variables](https://en.wikipedia.org/wiki/Gaussian_function#/media/File:Gaussian_2d_surface.png) está dada por la función:

$ w(x, y) = \frac{1}{2 . π σ^2} . e^{- \frac{x^2 + y^2}{2 . σ^2}} $


In [ ]:
# TODO: (gaussian blur)
# - escribir una función en python que dado ksize (el tamaño del kernel),
#   y sigma (el desvío estándard), devuelva el kernel gausiano resultante.
# - Visualizar el kernel como una imagen.
# - aplicar el kernel a la imagen, mostrar como varía el blur variando
#   los parámetros del kernel

import numpy as np

def gaussian_kernel(sigma=1, ksize=None, K=None):
  # ksize = 29
  # sigma = 3

  if ksize is None:
    # if ksize is None, ksize se
    # computará automáticamente con un tamaño de 2 * sigma + 1
    ksize = int(2 * sigma + 1)

  a = ksize // 2
  b = ksize // 2

  c = 2 * sigma**2
  if K is None:
    K = 1 / (c * np.pi)
  s = np.linspace(-b, b, ksize)
  t = np.linspace(-a, a, ksize)
  ss, tt = np.meshgrid(s, t)
  w = K * np.exp(-(ss**2 + tt**2) / c)
  return w


w = gaussian_kernel(ksize=5, sigma=1, K=1)

In [ ]:
w

In [ ]:
labels = []
sigmas = [1, 3, 7, 11]
kernels = []
k_labels = []
results = [img]
labels = ["original"]

border_type = cv2.BORDER_CONSTANT
# border_type = cv2.BORDER_REFLECT_101

for sigma in sigmas:

  k = gaussian_kernel(sigma=sigma)

  kernels.append(cv2.normalize(k, 0, 255, cv2.NORM_MINMAX))
  k_labels.append(f"kernel sigma:{sigma}")

  filtered = cv2.filter2D(
      img, kernel=k,
      ddepth=cv2.CV_64F,
      borderType=border_type
  )

  labels.append(f"sigma:{sigma}")
  results.append(filtered)


show_images(kernels, k_labels)
show_images(results, labels)


### Separabilidad

Un kernel 2D es separable si se puede escribir como dot product de dos kernels 1D.

$ w( x, y ) = w_x( x ) . w_y( y )^T $

**Ejemplo**

El kernel gaussiano es separable ya que:

$ w(x, y) = \frac{1}{2 . π σ^2} . e^{- \frac{x^2 + y^2}{2 . σ^2}} = gauss1D(x) . gauss1D(y)^T $

donde

$ gauss1D(x) = \frac{1}{\sqrt{2 . π σ^2}} . e^{- \frac{x^2}{2 . σ^2}} $


El efecto combinado de aplicar secuencialmente los dos kernels 1D en la imagen, es equivalente a aplicar el filtro 2D (el primero combinará pixeles vecinos en la dirección de las columnas y el segundo en la de las filas).

Si bien el resultado es el mismo, el tiempo computacional de aplicar dos filtros 1D vs aplicar 2D es menor.


#### Blurr en OpenCV con filtros separables

OpenCV provee la función `cv2.getGaussianKernel(ksize, sigma)`
que genera un kernel gaussiano de 1D.

Nota: esta función realiza un paso final de normalizacion, computando en realidad $ g( x ) = α * e^{-\frac{x^2}{ 2 . σ^2}} $
Con $ α $ un valor de escalado tal que $ \sum_i g(i) = 1$.



In [ ]:

# - usando la función cv2.getGaussianKernel generar el kernel gaussiano 2D
# - normalizar tal que la suma de los kernels sea 1 y comparar con el kernel
#    generado por la función codificada anteriormente



In [ ]:
def gaussian_kernel_1d(sigma=1, ksize=None, K=None):

  if ksize is None:
    # if ksize is None, ksize se
    # computará automáticamente con un tamaño de 2 * sigma + 1
    ksize = int(2 * sigma + 1)

  a = ksize // 2

  c = 2 * sigma**2
  if K is None:
    K = 1 / np.sqrt(c * np.pi)
  t = np.linspace(-a, a, ksize)
  w = K * np.exp(-(t**2) / c)

  w = w / w.sum()
  return w

In [ ]:
# calculo el kernel en 1d, y lo traspongo para que quede como vector columna
gauss1d = gaussian_kernel_1d(sigma=1, ksize=5).reshape(-1, 1)

In [ ]:
# obtengo el mismo kernel pero con la librería de OpenCV
cv_gauss1d = cv2.getGaussianKernel(sigma=1, ksize=5, ktype=cv2.CV_64F)

In [ ]:
cv_gauss1d

In [ ]:
# El resultado es el mismo
(gauss1d == cv_gauss1d).all()

In [ ]:
# usando los kernels 1d genero el gaussian kernel 2d
cv_gauss2d = np.dot(cv_gauss1d, cv_gauss1d.T)

In [ ]:
# genero el mismo kernel 2d con la funcion implementada anteriormente
gauss2d = gaussian_kernel(sigma=1, ksize=5)

In [ ]:
# normalizo ambos kernels 2d
gauss2d /= gauss2d.sum()
cv_gauss2d /= cv_gauss2d.sum()

In [ ]:
# chequeo que los kernels normalizados coinciden.
np.linalg.norm(gauss2d - cv_gauss2d).round(6)

In [ ]:
imshow(cv_gauss2d)

#### Aplicando kernels separables

OpenCV provee la función `cv2.sepFilter2D(img, ddepth, kernel_1dx, kernel_1dy)` que permite aplicar dos filtros 1d para lograr el mismo resultado que el filtro combinado 2d.



In [ ]:
# TODO sepFilter
# - aplicar a la imagen blurr usando el filtro 2d y luego los dos
# filtros 1d con la función cv2.sepFilter2D. Graficar los resultados y comparar

gauss_1d = cv2.getGaussianKernel(sigma=7, ksize=15)
gauss_2d = gaussian_kernel(sigma=7, ksize=15)

show_images([
  cv2.filter2D(img, cv2.CV_64F, gauss_2d),
  cv2.sepFilter2D(img, cv2.CV_64F, gauss_1d, gauss_1d),
],
[
  "blurred with filter2D",
  "blurred with sepFilter2D (1d + 1d)"
])



#### Determinar si un kernel es separable

Un kernel W 2D es separable si se puede escribir como el dot product de dos kernels 1D:

$ W(x, y) = W_x(x) . W_y(y) $

dado el kernel 2D W, cómo podemos saber si es separable?

#### Descomposición en Valores Singulares (SVD) - repaso

Dada la matriz $A \in \mathbb{R}^{m \times n}$, siempre existen 3 matrices $U$, $\Sigma$ y $V$ tales que A se puede descomponer como:

$$
A = U \Sigma V^T
$$

donde:
- $U \in \mathbb{R}^{m \times m}$. ortogonal. 
- $\Sigma \in \mathbb{R}^{m \times n}$. diagonal con valores no negativos.
- $V \in \mathbb{R}^{n \times n}$. ortogonal


##### Notas

- $\sigma_i = \Sigma_{i, i}$ son los valores singulares de A.
- el número de valores singulares distintos de cero coincide con el rango de $A$.
- Si se ordena decrecientemente según los sus valores singulares, la descomposición es única.
- forma compacta: $ A = \sum_{i=1}^r \sigma_i . u_i . v_i^t $

In [ ]:
# TODO is_seperable( k )
# Propiedad: un kernel es separable si el rango de sus columnas es 1.
# ( recordemos que el rango es el número de columnas linealmente independientes en esa matriz, que indican
#  la dimensión del espacio vectorial generado )
# Implementar una funcion en python que recibe un kernel 2d y decide si es o no separable

def rank( k ):

  u, s, vt = np.linalg.svd( k )
  # print( s)
  threshold = 1e-6
  rank = np.sum(s > threshold)

  return rank

In [ ]:
import cv2
import numpy as np

laplacian = np.array([
    [0, -1, 0], 
    [-1, 4, -1], 
    [0, -1, 0]
])

In [ ]:
rank( laplacian )

In [ ]:
print( "gauss separable?", rank(gauss2d) == 1)
print( "laplacian separable?", rank(laplacian) == 1 )

In [ ]:
import numpy as np

def separate_kernel(k):
    """
    Dado k un kernel 2D separable, lo separa en dos kernels de 1D.
    """
    
    # computa la descomposicion svd
    u, s, vt = np.linalg.svd(k)

    # Chequea si el kernel k es separable
    threshold = 1e-6
    if sum(s > threshold) != 1:
        raise ValueError("el kernel no es separable")

    # Extrae el primer valor singular
    # sigma = np.sqrt(s[0])
    sigma = s[0]

    # el producto externo entre la primera columna de u 
    # y de vt, escalado por el valor singular 
    # nos da los kernels 1D
    
    k_row = u[:, 0] * sigma
    k_col = vt[0, :]

    return k_row, k_col

k = np.array([
    [1, 0, -1], 
    [2, 0, -2], 
    [1, 0, -1]
])
kx, ky = separate_kernel( k )

In [ ]:
kx, ky

In [ ]:
np.dot(
    kx.reshape(-1, 1), 
    ky.reshape(1, -1)
)

## Blur con OpenCV

OpenCV provee la funcion `cv2.GaussianBlur( img, ksize, sigmaX [, borderType])` que realiza todo lo anterior de manera eficiente, es decir dada una imagen computa el kernel gaussiano y lo aplica y luego devuelve la correspondiente imagen blurreada.

En este caso ksize es una dupla que nos permite especificar un kernel con un tamaño y un desvío diferente en X y en Y.

In [ ]:
# Obs: los kernel gaussianos 2D, no tienen por qué ser cuadrados.
g1 = cv2.getGaussianKernel(sigma=1, ksize=3, ktype=cv2.CV_64F)
g2 = cv2.getGaussianKernel(sigma=1, ksize=5, ktype=cv2.CV_64F)
np.dot(g1, g2.T)

In [ ]:
# TODO:
# - aplicar blur sobre la imagen usando cv2.GaussianBlur y comparar los resultados
#.  con el ejercicio anterior.

blur_1 = cv2.GaussianBlur(img, ksize=(15, 15), sigmaX=7)
blur_2 = cv2.GaussianBlur(img, ksize=(15, 3), sigmaX=7, sigmaY=1)
blur_3 = cv2.GaussianBlur(img, ksize=(3, 15), sigmaX=1, sigmaY=7)
show_images([
    blur_1,
    blur_2,
    blur_3
], [
  "blurred sigma=7",
  "blurred sigma x=7, sigma y=1",
  "blurred sigma x=1, sigma y=7",
])


# Filtros Pasa alto

Dejan pasar elementos de alta frecuencia (como bordes, detalles finos y ruido) y atenúan los componentes de baja frecuencia (como regiones suaves y transiciones graduales).



In [ ]:
# img = cv2.imread("res/valve.jpg", cv2.IMREAD_GRAYSCALE)
img = cv2.imread("res/lenna.png", cv2.IMREAD_GRAYSCALE)


# Edge Detection


## Gradiente
En el procesamiento de imágenes, los bordes representan transiciones significativas de intensidad.

El gradiente de una imagen indica la tasa de cambio de intensidad en la imagen, y por tanto, los bordes corresponden a regiones con gradientes de mayor módulo.

El gradiente se compone de las derivadas parciales en cada dirección:

$ ∇I(x, y) = (G_x, G_y) = ( \frac{ \partial I }{\partial x}, \frac{ \partial I }{\partial y})$

Usando el cociente incremental

$ \frac{ \partial I }{\partial x} = \lim_{\Delta x \to 0} \frac{I(x + \Delta x, y) - I(x - \Delta x, y)}{\Delta x} $,

en la imagen discreta se puede aproximar como:

 $ \frac{ \partial I }{\partial x} ≈ \frac{I(x + 1, y) - I(x - 1, y)}{2} $


### Módulo del gradiente

El modulo del gradiente podemos calcularlo usando la norma L2:

$ | ∇I | = \sqrt{ G_x^2 + G_y^2 } $

### Orientación del gradiente

La orientación del gradiente coincide con el ángulo de orientación del borde y se puede calcular como:

$ arctan( \frac{ G_y }{ G_x } ) $

### Prewitt

El kernel Prewitt aproxima las derivadas de primer orden (gradientes) de la imagen tanto en la dirección horizontal (x) como en la vertical (y) usando los kernels:

$
G_x =
  \begin{bmatrix}
    -1 & 0 & 1 \\
    -1 & 0 & 1 \\
    -1 & 0 & 1
  \end{bmatrix} ★ I
$,

$
G_y =
  \begin{bmatrix}
    -1 & -1 & -1 \\
    0 & 0 & 0 \\
    1 & 1 & 1
  \end{bmatrix} ★ I
$


### Sobel

El filtro Sobel aproxima las derivadas de primer orden (gradientes) de la imagen tanto en la dirección horizontal (x) como en la vertical (y) usando los kernels:

$
G_x =
  \begin{bmatrix}
    -1 & 0 & 1 \\
    -2 & 0 & 2 \\
    -1 & 0 & 1
  \end{bmatrix}★ I
$,

$
G_y =
  \begin{bmatrix}
    -1 & -2 & -1 \\
    0 & 0 & 0 \\
    1 & 2 & 1
  \end{bmatrix} ★ I
$


### Scharr

El filtro Scharr aproxima las derivadas de primer orden (gradientes) de la imagen tanto en la dirección horizontal (x) como en la vertical (y) usando los kernels:

$
G_x =
  \begin{bmatrix}
    -3 & 0 & 3 \\
    -10 & 0 & 10 \\
    -3 & 0 & 3
  \end{bmatrix}★ I
$,

$
G_y =
  \begin{bmatrix}
    -3 & -10 & -3 \\
    0 & 0 & 0 \\
    3 & 10 & 3
  \end{bmatrix} ★ I
$


In [ ]:
# Gradiente

# - Aplicar filtros Prewitt, Sobel y Scharr para computar el gradiente de la imagen
# - Computar el módulo y la orientación del gradiente
# - Graficar el gradiente en cada dirección, el módulo y su orientación


import cv2

def compute_gradient(img, kx, ky):

  fimg = img.astype(float)
  fkx = kx.astype(float)
  fky = ky.astype(float)
  gx = cv2.filter2D(fimg, cv2.CV_64F, fkx)
  gy = cv2.filter2D(fimg, cv2.CV_64F, fky)

  mag = np.sqrt(gx**2 + gy**2)

  angle = cv2.phase(gx, gy, angleInDegrees=True)

  return gx, gy, mag, angle

def grad_orient_img(img, mag, angle):

  angle = angle / 2.0 # para convertir de 0:360 to 0:180

  h, w = img.shape
  hsv = np.zeros((h, w, 3), dtype='uint8')
  hsv[:, :, 0] = angle # H (en OpenCV entre 0 y 180 grados)
  hsv[:, :, 1] = 255 # S
  hsv[:, :, 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX) # V 0:255
  bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

  return bgr


prewitt = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
sobel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])

pw_gx, pw_gy, pw_mag, pw_angle = compute_gradient(img, prewitt, prewitt.T)
sb_gx, sb_gy, sb_mag, sb_angle = compute_gradient(img, sobel, sobel.T)

show_images([
    img,
    pw_gx,
    pw_gy,
], ["original", "Prewitt Gx", "Prewitt Gy"])

show_images([
    img,
    sb_gx,
    sb_gy,
], ["original", "Sobel Gx", "Sobel Gy"])

show_images([
    pw_mag,
    sb_mag
], ["Prewitt |G|", "Sobel |G|"])

show_images([
    grad_orient_img(img, pw_mag, pw_angle),
    grad_orient_img(img, sb_mag, sb_angle)
], ["Prewitt Orient(G)", "Sobel Orient(G)"])



## Gradiente usando OpenCV

OpenCV provee la funcion cv2.Sobel que realiza el mismo trabajo


In [ ]:
# - graficar Gx usando cv2.Sobel y cv2.filter2D
# - verificar que los resultados son los mismos

# devuelve la imagen "Sobeleada"
gx_cv = cv2.Sobel(img, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=3)

gx = cv2.filter2D(img, cv2.CV_64F, sobel)
show_images([
    gx_cv,
    gx
], ["Gx Sobel using CV", "Gx Sobel"]
)


In [ ]:
# - computar el módulo del gradiente usando Sobel y Scharr
# - graficar los resultado, y compararlos

def grad(img, method='sobel'):
  if method == 'sobel':
    gx = cv2.Sobel(img, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=3)
    gy = cv2.Sobel(img, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=3)
  else:
    # method == 'scharr'
    gx = cv2.Scharr(img, ddepth=cv2.CV_64F, dx=1, dy=0)
    gy = cv2.Scharr(img, ddepth=cv2.CV_64F, dx=0, dy=1)


  return gx, gy

img_blur = cv2.GaussianBlur(img, ksize=(15, 15), sigmaX=7)
sbx, sby = grad(img, 'sobel')
scx, scy = grad(img, 'scharr')


show_images([
    np.sqrt(sbx**2 + sby**2),
    np.sqrt(scx**2 + scy**2)
], ["Sobel X", "Scharr X"]
)

In [ ]:
# Roberts
# Otro Kernel que aproxima la primera derivada pero de 2x2

roberts_x = np.array([[-1, 0], [0, 1]])
roberts_y = np.array([[0, -1], [1, 0]])
gx = cv2.filter2D(img, cv2.CV_64F, roberts_x)
gy = cv2.filter2D(img, cv2.CV_64F, roberts_y)

mag = np.sqrt(gx**2 + gy**2)
show_images([
    gx,
    gy,
    mag
])


## Laplaciano

Corresponde con la derivadas parciales de segundo orden ("velocidad a la que cambian los valores de intensidad").

Se puede definir como:

$ ∇^2 I(x, y) = \frac{\partial^2 I}{\partial x^2} + \frac{\partial^2 I}{\partial y^2}$

(de esta manera se obtiene un kernel Isotrópico, donde no se favorece ninguna direccion en particular)


Si aproximamos la derivada como:

$ f'(x) ≈ \frac{f(x+1) - f(x - 1)}{2}$

La derivada segunda sería:

$ f''(x) ≈ \frac{f'(x+1) - f'(x - 1)}{2} = \frac{f(x+2) - 2f(x) + f(x - 2)}{4}$.

La fórmula anterior
- necesita mirar dos pixeles hacia adelante y dos hacia atrás
- si buscamos bordes, los mismos se van a encontrar en la misma posicion si divido por 4 que si divido por 1.

Entonces en la práctica se suele usar en cambio la siguiente aproximación a la derivada segunda:

$ f''(x) ≈ f(x + 1) - 2f(x) + f(x - 1)$.

Pasando a 2 variables tenemos:

$ \frac{\partial{^2I}}{\partial{x^2}}(x, y) ≈ f(x + 1, y) - 2f(x, y) + f(x - 1, y) $,

$ \frac{\partial{^2I}}{\partial{y^2}}(x, y) ≈ f(x, y + 1) - 2f(x, y) + f(y, y - 1) $.


Si utilizamos esta aproximación con x e y en la definicion del Laplaciano obtenemos:

$ ∇^2 I(x, y) = I(x+1, y) + I(x-1, y) + I(x, y+1) + I(x, y-1) - 4 I(x, y) $

El cómputo de esta ecuación se puede implementar usando una convolucion con el kernel:

$
L =
  \begin{bmatrix}
    0 & 1 & 0 \\
    1 & -4 & 1 \\
    0 & 1 & 0
  \end{bmatrix}
$,

Otros kernels posibles son:

$ L =
  \begin{bmatrix}
    1 & 1 & 1 \\
    1 & -8 & 1 \\
    1 & 1 & 1
  \end{bmatrix} 
  $

$ L = \frac{1}{6} .
  \begin{bmatrix}
    1 & 4 & 1 \\
    4 & -20 & 4 \\
    1 & 4 & 1
  \end{bmatrix} 
  $



In [ ]:
# El Laplaciano:
# - es isotrópico (responde igual a bordes en cualquier dirección) a diferencia de Sobel
# - es más sensible a ruido que Sobel, por lo que casi siempre se usa en combinación con suavizado.
# - los bordes están en los zero-crossings, a diferencia de Sobel en donde están en máximos y mínimos.

w_laplacian = np.array([
    [1, 1, 1],
    [1, -8, 1],
    [1, 1, 1],
])

# Podemos usar la convolución y uno de esos kernels para aplicar el laplaciano
laplacian = cv2.filter2D(img, cv2.CV_64F, w_laplacian)

# o bien usar la funcion de OpenCV cv2.Laplacian, que hace lo mismo
cv_laplacian = cv2.Laplacian(img, cv2.CV_64F, ksize=3)

laplacian_mod = np.absolute(laplacian)

show_images([
    laplacian,
    cv_laplacian,
    laplacian_mod,
], ["laplaciano usando kernel + filter2D", "usando cv.Laplacian", "modulo"])

### Sharpening con Laplaciano


El laplaciano resalta las transiciones de intensidad bruscas y resta importancia a las regiones de intensidades que varían lentamente. Para aumentar la nitidez de la imagen, se resta el laplaciano a la imagen:

$ S(x, y) = I (x, y) + c . [∇^2 I(x, y)] $

donde $ c $ es un número negativo ej $c=-1$.

OpenCV: nos brinda la funcion [`cv2.Laplacian`](https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gad78703e4c8fe703d479c1860d76429e6), que aplica el filtro laplaciano a una imagen.

In [ ]:
# TODO:
# mejorar la nitidez de la imagen "recursos/moon.jpg" mediante el
# realce de bordes con el filtro laplaciano.
# hint: puede requerir aumentar el contraste en el resultado final.

In [ ]:
def contraste(r, c1, c2, a):

  # transformacion de aumento de contraste
  # como una funcion partida por 3 rectas l0, l1 y l2
  # dados los valores de corte c1, c2, y a
  # siendo a la pendiente de l0 y l2

  # l0: 0 = a.0 + b => b = 0
  b0 = 0

  # l2: 255 = a. 255 + b2
  b2 = 255 * (1 - a)

  # calculo y1, y2
  y1 = a * c1 + 0
  y2 = a * c2 + b2

  # l1: y = a1 . r + b1
  # y1 = a1 . c1 + b1
  # y2 = a1 . c2 + b1
  # y1 - y2 = a1 . (c1 - c2)
  # b1 = y1 - a1 . c1
  a1 = (y1 - y2) / (c1 - c2)
  b1 = y1 - a1 * c1

  s0 = a * r + b0
  s1 = a1 * r + b1
  s2 = a * r + b2

  mask_0 = r < c1
  mask_1 = (r >= c1) & (r < c2)
  mask_2 = r >= c2

  s = s0 * mask_0 + s1 * mask_1 + s2 * mask_2
  return s

a = 1 / 4
c1 = 255 * 1 / 3
c2 = 255 * 2 / 3

In [ ]:
img = cv2.imread("res/moon.jpg", cv2.IMREAD_GRAYSCALE)


# w = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
w = np.array([[1, 1, 1], [1, -8, 1], [1, 1, 1]])

w = w.astype(float)
img = img.astype(float)

laplacian = cv_convolution(w, img)
laplacian = laplacian / np.max(np.abs(laplacian))

c = -255
sharpened = img + c * laplacian

sharpened = contraste(sharpened, 20, 220, 1/5)


show_images([
  img, sharpened
])

## Binarización de bordes

Binarización consiste en transformar una imagen a blanco y negro.
El resultado tiene dos valores posibles

Ejemplo: 0 o 1 (o 0 y 255).

In [ ]:
# EJERCICIO:

# Parte 1 - Detectar los bordes usando el gradiente y luego binarizar usando la
#   la funciónde OpenCV cv.threshold para obtener una imagen binaria con los bordes.

# img = cv2.imread("res/valve.jpg", 0)
img = cv2.imread("res/lenna.png", 0)



In [ ]:
# TODO:
# mag = "..."
# edges = "..."

gx, gy = grad(img)
mag = np.sqrt(gx**2 + gy**2)
edges = np.zeros_like(img)
edges[mag > 127] = 255

show_images([
    img, 
    mag,
    edges
], ["Original", "Modulo del Gradiente", "Edges"])


In [ ]:
# Parte 2. 
# Al obtener los edges con el gradiente obtenemos bordes fuertes y bordes suaves.
# Objetivo: Quiero una imagen binaria (0, 255), que decida si un pixel es borde o no.
# Problema: 
#   - Si hago threshold, me voy a quedar con los fuertes, pero pierdo los débiles.
#   - Si bajo el threshold se me mete ruido.

# TODO:
# Cómo podría mejorar este algoritmo para obtener mejores bordes en una imagen binaria?


In [ ]:
edges = cv2.Canny(
    img, 100, 200, 
)

In [ ]:
imshow(edges)